In [46]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

################
# PARSE FUNCS  #
################
def get_html(url):
    response = requests.get(url)
    return response.text


def get_common_table(page):
    soap = BeautifulSoup(page, "lxml")
    return soap.find('table', id='t_common')


def parse_headers(table):
    headers = table.find_all('tr')[1].find_all('td')
    return [header.text for header in headers]


def parse_rows(table):
    rows = table.find_all('tr')[2:]
    data = []
    for r in rows:
        rdata = []
        for td in r.find_all('td'):
            rdata.append(td.text.strip())
        data.append(rdata)
    return data


################
# PANDAS FUNCS #
################
def page_to_df(url):
    common_table = get_common_table(get_html(url))
    table, headers = parse_rows(common_table), parse_headers(common_table)
    df = pd.DataFrame(table, columns=headers)
    return df


def non_outsiders(df):
    return df[df['Примечание'] == '']


def only_originals(df):
    return df[df['Оригинал документа об образовании'] == 'да']


def add_institution_column(df, institution_name):
    df['Направление'] = institution_name
    return df.reset_index(drop=True)


def delete_user(df, id):
    user_index = df[df['id абитуриента/СНИЛС'] == id].index
    df = df.drop(user_index)
    return df


def get_passing_users_by_pr(df = None, pr = 1, count = 10):
    df = df.head(count)
    return df[df['Приоритет'] == f'{pr}']

In [63]:
def to_admite(directions = [], pr = 1):
    if len(directions) == 0:
        return ValueError('Нет направлений')
    if pr <= 0:
        return ValueError('Неправильный приоритет')
        
    user_list = []
    
    for d in directions:
        # берем верхушку с приоритетом pr
        free_places = d.places
        top_stdt = d.df.head(free_places)
        top_stdt_by_pr = top_stdt[top_stdt['Приоритет'] <= f'{pr}']

        # зачисляем
        d.admitted = pd.concat([d.admitted, top_stdt_by_pr])
        d.places -= top_stdt_by_pr.shape[0]
        # print(f'На {d.name} осталось мест: {d.places}, зачислили {top_stdt_by_pr.shape[0]} студентов с приоритетом {pr}')

        # запоминаем кто поступил
        user_list += top_stdt_by_pr['id абитуриента/СНИЛС'].to_list()

    for d in directions:
        d.df = d.df[~d.df['id абитуриента/СНИЛС'].isin(user_list)]
    
    return user_list


In [64]:
from settings import Direction, Tables

cfg = Tables()

# заполнение датафреймов
for i in cfg.get_directions():
    df = add_institution_column(non_outsiders(page_to_df(i.url)), f'{i.name}')
    i.df = df

stdt = pd.concat(cfg.get_dfs())
unique_stdt = stdt['id абитуриента/СНИЛС'].nunique()
print(f'Количество уникальных студентов В КОНКУРСЕ: {unique_stdt}')

# фильтр по оригам
for i in cfg.get_directions():
    df = only_originals(i.df)
    i.df = df

stdt = pd.concat(cfg.get_dfs())
unique_stdt = stdt['id абитуриента/СНИЛС'].nunique()
print(f'Количество уникальных студентов С ОРИГИНАЛАМИ: {unique_stdt}')

Количество уникальных студентов В КОНКУРСЕ: 261
Количество уникальных студентов С ОРИГИНАЛАМИ: 134


In [68]:
adm = 0
for p in range(1, 15):
    wave = to_admite(cfg.get_directions(), pr=p)
    # print(f'для приритета {p} зачислено {len(wave)}\n')
    adm += len(wave)
free = 0
for d in cfg.get_directions():
    free += d.places

print('осталось мест', free)

осталось мест 0


In [69]:
import pandas as pd

INN = '145-674-739-98'

def get_pos():
    res = pd.DataFrame()
    for df in cfg.get_admitted():
        filtered_df = df[df['id абитуриента/СНИЛС'] == INN]
        if not filtered_df.empty:
            res = filtered_df
            break
    return res if not res.empty else pd.DataFrame(columns=df.columns)

result = get_pos()
result


,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
30,31,145-674-739-98,75,нет,3,да,участвует в конкурсе,,PM


In [70]:
final_df = cfg.PMI.admitted.sort_values('Сумма конкурсных баллов')
final_df['Сумма конкурсных баллов'] = pd.to_numeric(final_df['Сумма конкурсных баллов'], errors='coerce')
final_df = final_df.sort_values('Сумма конкурсных баллов', ascending=False)
final_df = final_df.reset_index(drop=True)
final_df

,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
0,2,160-915-385-65,100,нет,1,да,участвует в конкурсе,,PMI
1,3,156-270-060-49,100,нет,1,да,участвует в конкурсе,,PMI
2,4,160-567-422-62,100,нет,1,да,участвует в конкурсе,,PMI
3,5,155-707-514-74,100,нет,1,да,участвует в конкурсе,,PMI
4,6,176-186-210-84,94,нет,1,да,участвует в конкурсе,,PMI
5,8,154-780-566-91,88,нет,1,да,участвует в конкурсе,,PMI
6,11,167-081-489-86,87,нет,1,да,участвует в конкурсе,,PMI
7,10,179-091-481-04,87,нет,1,да,участвует в конкурсе,,PMI
8,12,183-889-016-24,86,нет,3,да,участвует в конкурсе,,PMI
9,15,161-864-182-78,86,нет,1,да,участвует в конкурсе,,PMI


In [71]:
final_df = cfg.IST.admitted.sort_values('Сумма конкурсных баллов')
final_df['Сумма конкурсных баллов'] = pd.to_numeric(final_df['Сумма конкурсных баллов'], errors='coerce')
final_df = final_df.sort_values('Сумма конкурсных баллов', ascending=False)
final_df = final_df.reset_index(drop=True)
final_df

,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
0,2,199-797-837-94,99,нет,1,да,участвует в конкурсе,,IST
1,3,165-145-082-55,95,нет,1,да,участвует в конкурсе,,IST
2,7,151-020-047-81,89,нет,1,да,участвует в конкурсе,,IST
3,8,175-449-433-00,87,нет,1,да,участвует в конкурсе,,IST
4,9,142-816-090-49,86,нет,1,да,участвует в конкурсе,,IST
5,14,144-392-156-58,85,нет,1,да,участвует в конкурсе,,IST
6,15,144-339-705-63,85,нет,1,да,участвует в конкурсе,,IST
7,12,165-827-119-91,85,нет,1,да,участвует в конкурсе,,IST
8,11,164-857-305-99,85,нет,1,да,участвует в конкурсе,,IST
9,17,170-538-657-76,84,нет,1,да,участвует в конкурсе,,IST


In [72]:
final_df = cfg.PM.admitted.sort_values('Сумма конкурсных баллов')
final_df['Сумма конкурсных баллов'] = pd.to_numeric(final_df['Сумма конкурсных баллов'], errors='coerce')
final_df = final_df.sort_values('Сумма конкурсных баллов', ascending=False)
final_df = final_df.reset_index(drop=True)
final_df

,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
0,2,171-877-195-07,95,нет,1,да,участвует в конкурсе,,PM
1,3,151-755-180-61,92,нет,1,да,участвует в конкурсе,,PM
2,4,164-140-586-47,91,нет,1,да,участвует в конкурсе,,PM
3,6,155-703-263-58,90,нет,1,да,участвует в конкурсе,,PM
4,5,162-981-494-01,90,нет,1,да,участвует в конкурсе,,PM
5,7,154-717-380-76,90,нет,1,да,участвует в конкурсе,,PM
6,10,165-225-070-47,88,нет,1,да,участвует в конкурсе,,PM
7,13,179-870-737-43,86,нет,1,да,участвует в конкурсе,,PM
8,14,154-815-975-95,86,нет,1,да,участвует в конкурсе,,PM
9,19,147-749-796-31,83,нет,2,да,участвует в конкурсе,,PM


In [73]:
final_df = cfg.BI.admitted.sort_values('Сумма конкурсных баллов')
final_df['Сумма конкурсных баллов'] = pd.to_numeric(final_df['Сумма конкурсных баллов'], errors='coerce')
final_df = final_df.sort_values('Сумма конкурсных баллов', ascending=False)
final_df = final_df.reset_index(drop=True)
final_df

,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
0,1,162-711-587-59,91,нет,2,да,участвует в конкурсе,,BI
1,4,162-213-718-30,89,нет,2,да,участвует в конкурсе,,BI
2,9,139-576-704-08,88,нет,1,да,участвует в конкурсе,,BI
3,5,165-354-978-99,88,нет,2,да,участвует в конкурсе,,BI
4,7,055-320-350-21,88,нет,1,да,участвует в конкурсе,,BI
5,14,199-675-633-60,84,нет,1,да,участвует в конкурсе,,BI
6,16,152-610-482-33,81,нет,3,да,участвует в конкурсе,,BI
7,18,133-057-001-07,80,нет,1,да,участвует в конкурсе,,BI
8,19,159-763-431-13,80,нет,1,да,участвует в конкурсе,,BI
9,20,154-852-644-87,80,нет,2,да,участвует в конкурсе,,BI


In [74]:
final_df = cfg.FIIT.admitted.sort_values('Сумма конкурсных баллов')
final_df['Сумма конкурсных баллов'] = pd.to_numeric(final_df['Сумма конкурсных баллов'], errors='coerce')
final_df = final_df.sort_values('Сумма конкурсных баллов', ascending=False)
final_df = final_df.reset_index(drop=True)
final_df

,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
0,1,151-027-063-08,100,нет,1,да,участвует в конкурсе,,FIIT
1,2,189-241-265-94,100,нет,1,да,участвует в конкурсе,,FIIT
2,5,197-139-042-96,100,нет,1,да,участвует в конкурсе,,FIIT
3,7,161-185-415-48,100,нет,1,да,участвует в конкурсе,,FIIT
4,8,159-425-595-02,100,нет,1,да,участвует в конкурсе,,FIIT
5,12,162-300-625-15,100,нет,1,да,участвует в конкурсе,,FIIT
6,13,162-753-022-55,100,нет,2,да,участвует в конкурсе,,FIIT
7,4,118-116-162-24,100,нет,2,да,участвует в конкурсе,,FIIT
8,16,171-314-942-47,93,нет,1,да,участвует в конкурсе,,FIIT
9,21,159-375-530-04,85,нет,1,да,участвует в конкурсе,,FIIT


In [75]:
final_df = cfg.PI.admitted.sort_values('Сумма конкурсных баллов')
final_df['Сумма конкурсных баллов'] = pd.to_numeric(final_df['Сумма конкурсных баллов'], errors='coerce')
final_df = final_df.sort_values('Сумма конкурсных баллов', ascending=False)
final_df = final_df.reset_index(drop=True)
final_df

,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
0,7,155-941-709-91,100,нет,2,да,участвует в конкурсе,,PI
1,13,199-695-632-69,87,нет,1,да,участвует в конкурсе,,PI
2,16,152-336-135-33,83,нет,1,да,участвует в конкурсе,,PI
3,17,156-583-671-04,83,нет,1,да,участвует в конкурсе,,PI
4,28,156-450-615-64,77,нет,1,да,участвует в конкурсе,,PI
5,30,198-937-847-71,76,нет,2,да,участвует в конкурсе,,PI
6,31,176-910-876-08,76,нет,2,да,участвует в конкурсе,,PI
7,35,164-598-639-23,75,нет,2,да,участвует в конкурсе,,PI
8,43,158-550-781-97,73,нет,2,да,участвует в конкурсе,,PI
9,45,164-827-201-77,72,нет,1,да,участвует в конкурсе,,PI


In [76]:
final_df = cfg.IB.admitted.sort_values('Сумма конкурсных баллов')
final_df['Сумма конкурсных баллов'] = pd.to_numeric(final_df['Сумма конкурсных баллов'], errors='coerce')
final_df = final_df.sort_values('Сумма конкурсных баллов', ascending=False)
final_df = final_df.reset_index(drop=True)
final_df

,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
0,1,163-687-560-07,91,нет,1,да,участвует в конкурсе,,IB
1,3,154-750-695-84,89,нет,1,да,участвует в конкурсе,,IB
2,4,160-033-231-97,89,нет,1,да,участвует в конкурсе,,IB
3,5,157-743-184-94,88,нет,1,да,участвует в конкурсе,,IB
4,6,142-568-974-91,87,нет,2,да,участвует в конкурсе,,IB
5,7,160-690-769-79,86,нет,1,да,участвует в конкурсе,,IB
6,12,159-588-994-61,82,нет,1,да,участвует в конкурсе,,IB
7,15,145-778-121-92,80,нет,2,да,участвует в конкурсе,,IB
8,16,162-981-528-95,79,нет,2,да,участвует в конкурсе,,IB
9,17,164-552-938-88,78,да,1,да,участвует в конкурсе,,IB
